In [1]:
# https://www.databricks.com/blog/efficient-fine-tuning-lora-guide-llms
import pandas as pd
from datasets import load_dataset

/Users/tanay/Documents/Programming/calligraphy/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Run Inference w/ LoRA on Mistral7B

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, LoftQConfig, get_peft_model, PeftModel
from datasets import Dataset
import wandb

In [3]:
# can change the quantization type using
# https://huggingface.co/blog/4bit-transformers-bitsandbytes

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
base_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map='mps')

Loading checkpoint shards:  33%|███▎      | 1/3 [00:10<00:20, 10.45s/it]

In [11]:
# load existing adapter
model = PeftModel.from_pretrained(base_model, '../models/mistral_similes/textfx_adapter')

/Users/tanay/Documents/Programming/calligraphy/env/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


AssertionError: Torch not compiled with CUDA enabled

In [5]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer_function = lambda x : tokenizer(x['text'], truncation=True, padding='max_length', max_length=1024)

In [6]:
def run_inference(prompt):
     inputs = tokenizer(
          prompt,
          add_special_tokens=False,
          return_tensors="pt")["input_ids"]

     outputs = model.generate(
          inputs,
          max_length=100,
          do_sample=True,
          top_k=5,
          top_p=0.95,
          temperature=0.7,
          num_return_sequences=3)

     generated = tokenizer.decode(outputs[0])
     return generated

In [13]:
similes_test = 'A good simile contains a concrete image that illustrates the concept we want to convey without being too obvious. Good similes are unexpected and evocative. Create a simile that illustrates this concept: hunger.'
run_inference(similes_test)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'A good simile contains a concrete image that illustrates the concept we want to convey without being too obvious. Good similes are unexpected and evocative. Create a simile that illustrates this concept: hunger.\n\nA: The hunger gnaws at my insides like a wild beast.\nB: My stomach growls like a lion, reminding me of its empty lair.\nC: My hunger is a ravenous beast, devouring my thoughts and'